In [0]:
#t SNE plot of image embeddings

from sklearn.manifold import TSNE
from tensorflow.python.keras.preprocessing import image
from scipy.spatial.distance import cdist
import pandas as pd
import matplotlib.pyplot as plt



image_names_1 = image_names[0:6400]
image_300_vectors_1 = image_300_vectors[0:6400]

img_size = 50
path = 'Flicker8k_Dataset/'
img_collection = []
for img in image_names_1:
  img_collection.append(image.load_img(path + img, target_size=(img_size, img_size)))
  
import time
from lapjv import lapjv
#Plotting t-SNE of embeddings in a grid
#https://github.com/prabodhhere/tsne-grid/blob/master/tsne_grid.py

start =time.time()
def generate_tsne(activations):
    tsne = TSNE(perplexity=50, n_components=2, init='random', n_iter=250)
    X_2d = tsne.fit_transform(np.array(activations)[0:6400,:])
    X_2d -= X_2d.min(axis=0)
    X_2d /= X_2d.max(axis=0)
    return X_2d
print(time.time() -start)


X_2d = generate_tsne(image_300_vectors_1)  

def save_tsne_grid(img_collection, X_2d, out_res, out_dim):
    grid = np.dstack(np.meshgrid(np.linspace(0, 1, out_dim), np.linspace(0, 1, out_dim))).reshape(-1, 2)
    cost_matrix = cdist(grid, X_2d, "sqeuclidean").astype(np.float32)
    cost_matrix = cost_matrix * (100000 / cost_matrix.max())
    row_asses, col_asses, _ = lapjv(cost_matrix)
    grid_jv = grid[col_asses]
    out = np.ones((out_dim*out_res, out_dim*out_res, 3))

    for pos, img in zip(grid_jv, img_collection[0:out_dim**2]):
        h_range = int(np.floor(pos[0]* (out_dim - 1) * out_res))
        w_range = int(np.floor(pos[1]* (out_dim - 1) * out_res))
        out[h_range:h_range + out_res, w_range:w_range + out_res]  = image.img_to_array(img)

    im = image.array_to_img(out)
    im.save('t-sne_embeddings.png', quality=100)
save_tsne_grid(img_collection,X_2d,50,80)

In [0]:
#Word t-SNE

tsne = TSNE(perplexity=10, n_components=2, init='random', n_iter=250)
X_tsne = tsne.fit_transform(np.array(word_300_vectors))


with open("tensors.tsv", 'w+') as file_metadata:
    for i in range(len(word_list)):
        vector_row = '\t'.join(map(str, np.array(word_300_vectors)[i]))
        file_metadata.write(vector_row + '\n')

In [0]:
##Iterations vs images for caption to image query
from tensorflow.python.keras.preprocessing import image

image_names_1 = similar_images(n[0], image_300_vectors, image_names)

path = 'Flicker8k_Dataset/'
img_collection = []
for img in image_names_1:
  img_collection.append(np.uint8(image.img_to_array(image.load_img(path + img, target_size=(100, 100)))))
  

fig=plt.figure(figsize=(10, 1))
      
for i in range(1, 11):          
    img_1 = img_collection[i-1]          
    fig.add_subplot(1, 10, i)          
    plt.imshow(img_1)
    plt.xticks([])
    plt.yticks([])
plt.show()
